In [2]:
from gpcam import GPOptimizer

import numpy as np
from numpy.random import default_rng
import pandas as pd

import matplotlib.pyplot as plt
import plotly.graph_objects as go

import os
import csv

import random

from scipy.ndimage import gaussian_filter1d
from scipy.stats import entropy


In [3]:
energy_data = np.load("/data/Synthetic Data Generation_2/my_synthetic_energy.npy")
cycle_number = np.load("/data/Synthetic Data Generation_2/my_synthetic_cycleNum.npy")


label_size = 30

num_of_datasets = 25
considered_batteries = np.array([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24])


plt.figure(figsize = (20,10))
for i in considered_batteries: plt.scatter(cycle_number,energy_data[int(i)])
plt.tick_params(axis='both', which='major', labelsize=label_size) # Set the font size of the tick labels on the x and y axes
plt.xlabel("Cycle Number",fontsize=label_size)
plt.ylabel("Quantity of Interest",fontsize=label_size)
plt.show()


print("max y: ", np.max(energy_data))

# All Data
x_data_all = np.tile(cycle_number, (num_of_datasets, 1)).reshape(-1, 1) # repeat cycle 20 times to create x_data
y_data_all = np.vstack(energy_data[considered_batteries, :]).reshape(-1, 1)
x_pred = np.linspace(0,2000,1001).reshape(-1,1)

print("x data: ", x_data_all.shape)
print("y data: ", y_data_all.shape)
           
#plt.scatter(x_data,y_data)

max y:  499.8267959249855
x data:  (1275, 1)
y data:  (1275, 1)


# Creating the Subfolder in Results

In [4]:
# Specify the path for the new folder
new_folder_path = f"/results/Stopping Criterion/With Stopping/"

# Create the folder
os.makedirs(new_folder_path, exist_ok=True)




# Defining the GP Components

In [12]:
def get_distance_matrix(x1,x2):
    d = np.zeros((len(x1),len(x2)))
    for i in range(x1.shape[1]):
        d += (x1[:,i].reshape(-1, 1) - x2[:,i])**2
    return np.sqrt(d)


def my_noise(x,hps,obj):

    my_slope     = hps[2]
    my_pow       = hps[3]
    my_intercept = hps[4]

    my_s = my_slope * x**my_pow + my_intercept
    noise = np.diag(my_s[:,0])

    return noise


def kernel(x1,x2,hps,obj):
    d = get_distance_matrix(x1,x2) 

    k = hps[0] * obj.squared_exponential_kernel(d,hps[1]) 
    return k

# here I am assuming that the mean function is a piecewise function
def mean2(x,hps,obj):

    x0 = hps[5]

    m1 = hps[6]
    m2 = hps[7]
    
    b1 = obj.args[0]
    b2 = (m1 - m2) * x0 + b1

    y = np.where(x[:,0] <= x0, m1*x[:,0] + b1, m2*x[:,0] + b2)
    return y

# Creating Distribution

In [13]:
def make_distribution(m,v,x_pred):
    distr = np.zeros((len(x_pred),1000))
    y = np.linspace(0,500,1000)
    std = np.sqrt(v)
    for i in range(len(x_pred)):
        distr[i] = 1./(std[i] * np.sqrt(2 * np.pi)) * np.exp( - (y - m[i])**2 / (2. * std[i]**2))
    return distr

# Finding Ground Truth Probability of Failure

In [14]:
init_hyperparameters = np.array([150, 250,                # Kernel
                                  0.06, 2, 2,             # Noise  
                                  250,-0.01,-0.015])      # Mean.


my_trained_hps = np.array([1.00737232e+02,  3.98339991e+02,  1.00005243e-05,  2.54249162e+00,
                           1.84718191e+00,  5.14932171e+02, -4.76211299e-02, -9.36965609e-02])

dx = 1000./1000.
    

# Setting the Optimization Bounds for Hyperparameters
bounds = np.empty((8,2))
# Kernel Sq Exp 
bounds[0] = np.array([100.,10000.])                         # Kernel Variance
bounds[1] = np.array([10.,400.])                           # Kernel Lengthscale
# Noise
bounds[2] = np.array([1e-5,1.])                            # Noise Slope
bounds[3] = np.array([1.,5.])                              # Noise Power
bounds[4] = np.array([0.,3.])                              # Noise Intercept
# Mean
bounds[5] = np.array([200.,700.])                          # Mean Piecewise Intersection point
bounds[6] = np.array([-1e-1,-1e-3])                        # Mean Slope 1
bounds[7] = np.array([-5e-1,-1e-3])                        # Mean Slope 2

    
# Finding the mean of the initial capacity to be entered to the code
my_ind = np.where(x_data_all<=10)
initial_capacity = np.mean(y_data_all[my_ind[0]])
    
my_gpGT = GPOptimizer(x_data_all,y_data_all,
            init_hyperparameters = my_trained_hps,  # we need enough of those for kernel, noise and prior mean functions
            #noise_variances=np.ones(y_data.shape) * 0.01, #provding noise variances and a noise function will raise a warning 
            compute_device='cpu', 
            gp_kernel_function=kernel, 
            gp_kernel_function_grad=None, 
            gp_mean_function=mean2, 
            gp_mean_function_grad=None,
            gp_noise_function=my_noise,
            normalize_y=False,
            sparse_mode=False,
            gp2Scale = False,
            store_inv=False, 
            ram_economy=False, 
            args=np.array([initial_capacity]))

 
#my_gpGT.train(hyperparameter_bounds=bounds, method='global')

my_GT_hps = my_gpGT.hyperparameters

print("GT Training Done!")

GT Training Done!


In [15]:
print("GT Hyperparameters: ", my_gpGT.hyperparameters)

x_pred = np.linspace(0,2000,1001).reshape(-1,1)

f_GT = my_gpGT.posterior_mean(x_pred.reshape(-1,1))["f(x)"]
v_GT = my_gpGT.posterior_covariance(x_pred.reshape(-1,1), variance_only=False, add_noise=True)["v(x)"]


# Plotting the data
plt.figure(figsize = (15,10))
plt.plot(x_pred[:,0],f_GT, color = "blue", linewidth = 3,label='Fitted Model')
plt.scatter(x_data_all,y_data_all, color = "black",label='Data Points') # Training Data
plt.fill_between(x_pred[:,0],f_GT - 2. * np.sqrt(v_GT), f_GT + 2. * np.sqrt(v_GT), alpha = 0.5, color = "grey", label='Confidence Interval')
plt.xlabel("Cycle Number",fontsize=label_size) 
plt.ylabel("Energy",fontsize=label_size)
plt.ylim(380,510)
#plt.yticks([130, 180, 230, 280, 330, 380])
plt.xlim(0,1000)
plt.legend(fontsize=label_size,frameon=False,loc='lower left')
plt.tick_params(axis='both', which='major', labelsize=label_size) # Set the font size of the tick labels on the x and y axes
plt.show()

GT Hyperparameters:  [ 1.00737232e+02  3.98339991e+02  1.00005243e-05  2.54249162e+00
  1.84718191e+00  5.14932171e+02 -4.76211299e-02 -9.36965609e-02]


In [17]:

d_GT = make_distribution(f_GT,v_GT,x_pred)[:,900].T

d_GT = d_GT/(np.sum(d_GT) * dx)


# Number cycles where 5% of batteries failed 
cum_distr = np.cumsum(d_GT)*dx
index_95failed = np.argmin(abs(cum_distr - 0.05))
print("95 percent failed after: ", x_pred[index_95failed,0])
        
#print(d.shape,x_pred[:,0].shape)
plt.figure(figsize = (15,10))
plt.plot(x_pred, d_GT,linewidth = 3)
#plt.xlabel("cycle number")
#plt.ylabel("p")
plt.fill_between(x_pred[0:index_95failed, 0], d_GT[0:index_95failed], alpha=0.4, color="red", label='Area under Curve')
plt.savefig('/results/Stopping Criterion/With Stopping/POF_my_plot_GT.png') # saving plot with a unique name 

95 percent failed after:  642.0


# __________________________________________________________________
# Sequence of Experiments 

In [18]:
# Here I am assuming that I am doing experiments sequentially and I am calculating the total correlation of the new experiments

init_hyperparameters = my_GT_hps

# Setting the Optimization Bounds for Hyperparameters
bounds = np.empty((8,2))
# Kernel Sq Exp 
bounds[0] = np.array([100.,10000.])                         # Kernel Variance
bounds[1] = np.array([10.,400.])                           # Kernel Lengthscale
# Noise
bounds[2] = np.array([1e-5,1.])                            # Noise Slope
bounds[3] = np.array([1.,5.])                              # Noise Power
bounds[4] = np.array([0.,3.])                              # Noise Intercept
# Mean
bounds[5] = np.array([200.,700.])                          # Mean Piecewise Intersection point
bounds[6] = np.array([-1e-1,-1e-3])                        # Mean Slope 1
bounds[7] = np.array([-5e-1,-1e-3])                        # Mean Slope 2


label_size = 30
# Change this value for different threshold cases, in the paper we have [0.1, 1]
error_threshold = 0.5

num_versions = 30


# Set the random seed for reproducibility
random_seed = 42
# Generate 10 different random sequences
np.random.seed(random_seed)  # Set the seed outside the loop
random_sequences = [np.random.permutation(num_of_datasets) for _ in range(num_versions)]



x_data_all = np.tile(cycle_number, (num_of_datasets, 1)).reshape(-1, 1) # repeat cycle 20 times to create x_data
x_pred = np.linspace(0,2000,1001).reshape(-1,1)



for t in range(0, 1):#num_versions):
    
    
    # My condition to switch off the training of the hps to save on computing power
    condition_for_Training = True 

    y_data_all = np.vstack(energy_data[random_sequences[t], :]).reshape(-1, 1)

    
    # Specify the path for the new folder
    new_folder_path = f"/results/Stopping Criterion/With Stopping/Trial_{t}"

    # Create the folder
    os.makedirs(new_folder_path, exist_ok=True)

    %cd {new_folder_path}
    
    # Check if the csv files exist and delete them if present
    if os.path.exists(f"/results/Stopping Criterion/With Stopping/Trial_{t}/Failure_Cycle_Number.csv"): os.remove(f"/results/Stopping Criterion/With Stopping/Trial_{t}/Failure_Cycle_Number.csv")
    if os.path.exists(f"/results/Stopping Criterion/With Stopping/Trial_{t}/Trained Hyperparameters.csv"): os.remove(f"/results/Stopping Criterion/With Stopping/Trial_{t}/Trained Hyperparameters.csv")    
    if os.path.exists(f"/results/Stopping Criterion/With Stopping/Trial_{t}/Total Correlation.csv"): os.remove(f"/results/Stopping Criterion/With Stopping/Trial_{t}/Total Correlation.csv")
    if os.path.exists(f"/results/Stopping Criterion/With Stopping/Trial_{t}/Average KL Values and Threshold.csv"): os.remove(f"/results/Stopping Criterion/With Stopping/Trial_{t}/Average KL Values and Threshold.csv")

    x_data = np.empty((0, 1))
    y_data = np.empty((0, 1))

    my_counter = 0
    my_calc_kl = np.empty((0, 1))

    for num_of_batteries_GP_fitting in range(0,25): # The number of batteries loop!
    
        print(num_of_batteries_GP_fitting)
        
        for j in range(0,50,1): # The number of data points per battery loop!
   
            if j % 2 == 0:
               print(j)  
     
            x_data = np.append(x_data,x_data_all[num_of_batteries_GP_fitting*51 + j: num_of_batteries_GP_fitting*51 + j +1])
            y_data = np.append(y_data,y_data_all[num_of_batteries_GP_fitting*51 + j: num_of_batteries_GP_fitting*51 + j +1])
    
            # Finding the mean of the initial capacity to be entered to the code
            my_ind = np.where(x_data<=10)
            initial_capacity = np.mean(y_data[my_ind[0]])
    
            my_gp1 = GPOptimizer(x_data,y_data,
                                 init_hyperparameters = init_hyperparameters,  # we need enough of those for kernel, noise and prior mean functions
                                 #noise_variances=np.ones(y_data.shape) * 0.01, #provding noise variances and a noise function will raise a warning 
                                 compute_device='cpu', 
                                 gp_kernel_function=kernel, 
                                 gp_kernel_function_grad=None, 
                                 gp_mean_function=mean2, 
                                 gp_mean_function_grad=None,
                                 gp_noise_function=my_noise,
                                 normalize_y=False,
                                 sparse_mode=False,
                                 gp2Scale = False,
                                 store_inv=False, 
                                 ram_economy=False, 
                                 args=np.array([initial_capacity]))

            if condition_for_Training:
                
                my_gp1.train(hyperparameter_bounds=bounds, method='global')
            
      
            # Saving Trained Hyperparameters
            row_hyperparameters = [my_counter, j] +  list(my_gp1.hyperparameters)
        
            # Write the row_entropies to the CSV file
            csv_file = open(f"/results/Stopping Criterion/With Stopping/Trial_{t}/Trained Hyperparameters.csv", 'a', newline='')
            csv_writer = csv.writer(csv_file)
            csv_writer.writerow(row_hyperparameters)
            csv_file.close() # Close the CSV file
        
        
            f = my_gp1.posterior_mean(x_pred)["f(x)"]
            v =  my_gp1.posterior_covariance(x_pred, variance_only=False, add_noise=True)["v(x)"]
            vnn =  my_gp1.posterior_covariance(x_pred, variance_only=False, add_noise=False)["v(x)"]      
             
            # Plotting the data
            plt.figure(figsize = (15,10))
            plt.plot(x_pred[:,0],f, color = "blue", linewidth = 5,label='Fitted Model')
            plt.scatter(x_data,y_data, s=250, color = "black",label='Data Points') # Training Data
            plt.fill_between(x_pred[:,0],f - 2. * np.sqrt(v), f + 2. * np.sqrt(v), alpha = 0.4, color = "grey", label='Confidence Interval')
            plt.xlabel("Cycle Number",fontsize=label_size) 
            plt.ylabel("Energy",fontsize=label_size)
            plt.ylim(380,510)
            plt.yticks([380,410, 440, 470, 500])
            plt.xlim(0,1000)
            plt.legend(fontsize=label_size,frameon=False,loc='lower left')
            plt.tick_params(axis='both', which='major', labelsize=label_size) # Set the font size of the tick labels on the x and y axes

            ## Save and close the plot
            plt.savefig(f"/results/Stopping Criterion/With Stopping/Trial_{t}/my_plot_{my_counter}.png") # saving plot with a unique name 
            plt.close() # closing the plot to free up memory
            #plt.show()
        
        
            #########################################################################
            #########################################################################
            #########################################################################
        
            # To check if I stop current battery

        
            my_trained_hps = my_gp1.hyperparameters
        
            x_data_current_battery = np.array(x_data_all[num_of_batteries_GP_fitting*51: num_of_batteries_GP_fitting*51 + j +1])
            y_data_current_battery = np.array(y_data_all[num_of_batteries_GP_fitting*51: num_of_batteries_GP_fitting*51 + j +1])

        
            my_gp2 = GPOptimizer(x_data_current_battery,y_data_current_battery,
                                 init_hyperparameters = my_trained_hps,  # we need enough of those for kernel, noise and prior mean functions
                                 #noise_variances=np.ones(y_data.shape) * 0.01, #provding noise variances and a noise function will raise a warning 
                                 compute_device='cpu', 
                                 gp_kernel_function=kernel, 
                                 gp_kernel_function_grad=None, 
                                 gp_mean_function=mean2, 
                                 gp_mean_function_grad=None,
                                 gp_noise_function=my_noise,
                                 normalize_y=False,
                                 sparse_mode=False,
                                 gp2Scale = False,
                                 store_inv=False, 
                                 ram_economy=False, 
                                 args=np.array([initial_capacity]))
     
    
            ####################################################
            x_new_lower_limit = x_data_all[num_of_batteries_GP_fitting*51 + j + 1 ]
            x_pred2 = np.linspace(x_new_lower_limit,1000,50).reshape(-1,1)
    
            my_posterior_mean_CurrentBattery = my_gp2.posterior_mean(x_pred2)["f(x)"]
            my_posterior_covariance_CurrentBattery = my_gp2.posterior_covariance(x_pred2)['S'] + 1e-7 * np.eye(len(x_pred2))
        
        
            my_posterior_mean_AllData = my_gp1.posterior_mean(x_pred2)["f(x)"]
            my_posterior_covariance_AllData = my_gp1.posterior_covariance(x_pred2)['S'] + 1e-7 * np.eye(len(x_pred2))

            diff_CurrentBattery_AllData = my_gp1.kl_div(my_posterior_mean_CurrentBattery,my_posterior_mean_AllData,my_posterior_covariance_CurrentBattery,my_posterior_covariance_AllData)
            #print("KL Divergence for Battery: ",diff_CurrentBattery_AllData)

            ########################################################################

            # Create a row with the extracted values
            row_entropies = [my_counter, num_of_batteries_GP_fitting, j, diff_CurrentBattery_AllData]#, my_total_correlation1]

            # Write the row_entropies to the CSV file
            csv_file = open(f"/results/Stopping Criterion/With Stopping/Trial_{t}/Total Correlation.csv", 'a', newline='')
            csv_writer = csv.writer(csv_file)
            csv_writer.writerow(row_entropies)
            csv_file.close() # Close the CSV file


            #########################################################################
            #########################################################################
            #########################################################################
        
            # Calculate Probability of Failure Distribution and finding the cycle at which 5% failed <90% initial energy 

            d = make_distribution(f,v,x_pred)[:,900].T

            d = d/(np.sum(d) * dx)
   
        
            # Number cycles where 5% of batteries failed 
            cum_distr = np.cumsum(d)*dx
            index_95failed = np.argmin(abs(cum_distr - 0.05))

            # Calculating the KL divergence between the current probability of failure and the GT probability of failrue
            my_kl_between_pof = entropy(d, d_GT)
        
            row_failures = [my_counter, num_of_batteries_GP_fitting, j, x_pred[index_95failed,0], my_kl_between_pof]
            csv_file = open(f"/results/Stopping Criterion/With Stopping/Trial_{t}/Failure_Cycle_Number.csv", 'a', newline='')
            csv_writer = csv.writer(csv_file)
            csv_writer.writerow(row_failures)
            csv_file.close() # Close the CSV file
        
        
            if my_kl_between_pof <= 0.05:
                print("No Training")
                condition_for_Training = False

        
            my_counter +=1

        
            #########################################################################
            #########################################################################
            #########################################################################
        
            # break the inner loop -- Stop current battery experimentation if not informative
        
            if num_of_batteries_GP_fitting < 2:
                continue
                  
            my_calc_kl = np.append(my_calc_kl,diff_CurrentBattery_AllData)
    
            # Calculate the cumulative sum up to the current index
            cumulative_sum = np.sum(my_calc_kl)
            my_thresholded_average = error_threshold * cumulative_sum / len(my_calc_kl)
        
            row_averages = [my_counter-1, num_of_batteries_GP_fitting, j , my_thresholded_average]
        
            csv_file = open(f"/results/Stopping Criterion/With Stopping/Trial_{t}/Average KL Values and Threshold.csv", 'a', newline='')
            csv_writer = csv.writer(csv_file)
            csv_writer.writerow(row_averages)
            csv_file.close() # Close the CSV file
        
            # Check if the current my_kl_filtered value is larger than the error*threshold, do not consider the first 7 values of each battery
            if j<7:
                continue
        
            if my_calc_kl[-1] < my_thresholded_average:
                print("Current battery is not informative, it is SKIPPED")
                break  # Break the current loop

    
            #########################################################################
            #########################################################################
            #########################################################################
     
    
        print("===========================")
    
    %cd ..

    print("Training Complete for All Batteries for this trial")



/opt/conda/lib/python3.9/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


/results/Stopping Criterion/With Stopping/Trial_0
0
0
2
4
6
8
10
12
14
16
18
20
22
24
26
28
30
32
34
36
38
40
42
44
46
48
1
0
2
4
6
8
10
12
14
16
18
20
22
24
26
28
30
32
34
36
38
40
42
44
46
48
2
0
2
4
6
8
10
12
Current battery is not informative, it is SKIPPED
3
0
2
4
6
8
10
12
14
16
18
20
Current battery is not informative, it is SKIPPED
4
0
2


KeyboardInterrupt: 

No Training
No Training
36
No Training
No Training
38
No Training
No Training
40
No Training
42
44
46
48
10
0
2
4
6
8
10
12
14
16
18
20
22
24
26
28
30
32
34
36
38
40
42
44
46
48
Current battery is not informative, it is SKIPPED
11
0
2
4
6
8
10
12
14
16
18
20
22
24
26
28
30
32
34
36
38
40
42
44
46
48
12
0
2
4
6
8
10
12
14
16
18
20
22
24
26
28
30
32
34
36
38
40
42
44
46
48
Current battery is not informative, it is SKIPPED
13
0
2
4
6
8
10
12
14
16
18
20
22
24
26
28
30
32
34
36
38
40
42
44
46
48
14
0
2
4
6
8
10
12
14
16
18
20
22
24
26
28
30
32
34
36
38
40
42
44
46
48
15
0
2
4
6
8
10
12
14
16
18
20
22
No Training
No Training
24
No Training
No Training
26
No Training
No Training
28
No Training
No Training
30
No Training
No Training
32
No Training
No Training
34
No Training
No Training
36
No Training
No Training
38
No Training
No Training
40
No Training
No Training
42
No Training
No Training
44
No Training
No Training
46
No Training
No Training
48
No Training
No Training
16
0
No Training
No T

28
30
32
34
36
38
40
Current battery is not informative, it is SKIPPED
4
0
2
4
6
8
10
12
No Training
14
No Training
No Training
16
No Training
No Training
18
No Training
No Training
20
No Training
No Training
22
No Training
No Training
24
No Training
26
28
30
32
34
36
38
40
42
44
46
48
Current battery is not informative, it is SKIPPED
5
0
2
4
6
8
10
12
14
16
18
20
22
24
26
28
30
32
34
36
38
40
42
44
46
48
6
0
2
4
6
8
10
12
14
16
18
20
22
24
26
28
30
32
34
36
38
40
No Training
No Training
42
No Training
No Training
44
No Training
No Training
46
No Training
No Training
48
No Training
No Training
Current battery is not informative, it is SKIPPED
7
0
No Training
No Training
2
No Training
No Training
4
No Training
No Training
6
No Training
No Training
8
No Training
No Training
10
No Training
No Training
12
No Training
No Training
14
No Training
No Training
16
No Training
No Training
18
No Training
No Training
20
No Training
No Training
22
No Training
No Training
24
No Training
No Training
2

...

No Training
No Training
26
No Training
No Training
28
No Training
No Training
30
No Training
No Training
32
No Training
No Training
34
No Training
No Training
36
No Training
No Training
38
No Training
No Training
40
No Training
No Training
42
No Training
No Training
44
No Training
No Training
46
No Training
No Training
48
No Training
No Training
19
0
No Training
No Training
2
No Training
No Training
4
No Training
No Training
6
No Training
No Training
8
No Training
No Training
10
No Training
No Training
12
No Training
No Training
14
No Training
No Training
16
No Training
No Training
18
No Training
No Training
20
No Training
No Training
22
No Training
No Training
24
No Training
No Training
26
No Training
No Training
28
No Training
No Training
30
No Training
No Training
32
No Training
No Training
34
No Training
No Training
36
No Training
No Training
38
No Training
No Training
40
No Training
No Training
42
No Training
No Training
44
No Training
No Training
46
No Training
No Training
Curren

No Training
No Training
36
No Training
No Training
38
No Training
No Training
40
No Training
No Training
42
No Training
No Training
44
No Training
No Training
46
No Training
No Training
48
No Training
No Training
11
0
No Training
No Training
2
No Training
No Training
4
No Training
No Training
6
No Training
No Training
8
No Training
No Training
10
No Training
No Training
12
No Training
No Training
14
No Training
No Training
16
No Training
No Training
18
No Training
No Training
20
No Training
No Training
22
No Training
No Training
24
No Training
No Training
26
No Training
No Training
28
No Training
No Training
30
No Training
No Training
32
No Training
No Training
34
No Training
No Training
36
No Training
No Training
38
No Training
No Training
40
No Training
No Training
42
No Training
No Training
44
No Training
No Training
46
No Training
No Training
48
No Training
No Training
12
0
No Training
No Training
2
No Training
No Training
4
No Training
No Training
6
No Training
No Training
8
No Tr

No Training
No Training
18
No Training
No Training
20
No Training
No Training
22
No Training
No Training
24
No Training
No Training
26
No Training
No Training
28
No Training
No Training
30
No Training
No Training
32
No Training
No Training
34
No Training
No Training
36
No Training
No Training
38
No Training
No Training
40
No Training
No Training
42
No Training
No Training
44
No Training
No Training
46
No Training
No Training
48
No Training
No Training
23
0
No Training
No Training
2
No Training
No Training
4
No Training
No Training
6
No Training
No Training
8
No Training
No Training
10
No Training
No Training
12
No Training
No Training
14
No Training
No Training
16
No Training
No Training
18
No Training
No Training
20
No Training
No Training
22
No Training
No Training
24
No Training
No Training
26
No Training
No Training
28
No Training
No Training
30
No Training
No Training
32
No Training
No Training
34
No Training
No Training
36
No Training
No Training
38
No Training
No Training
40
No 

No Training
10
No Training
No Training
12
No Training
No Training
14
No Training
No Training
16
No Training
No Training
18
No Training
No Training
20
No Training
No Training
22
No Training
No Training
24
No Training
No Training
26
No Training
No Training
28
No Training
No Training
30
No Training
No Training
32
No Training
No Training
34
No Training
No Training
36
No Training
Current battery is not informative, it is SKIPPED
19
0
No Training
No Training
2
No Training
No Training
4
No Training
No Training
6
No Training
No Training
8
No Training
No Training
10
No Training
No Training
12
No Training
No Training
14
No Training
No Training
16
No Training
No Training
18
No Training
No Training
20
No Training
No Training
22
No Training
No Training
24
No Training
No Training
26
No Training
No Training
28
No Training
No Training
30
No Training
No Training
32
No Training
No Training
34
No Training
No Training
36
No Training
No Training
38
No Training
No Training
40
No Training
No Training
42
No T

MemoryError: Unable to allocate 8.93 MiB for an array with shape (1169, 1001) and data type float64